In [1]:
# Minimal production: PT(YJ) + LogisticRegression (L2, C=2000) 5-fold OOF + submission
import time, random, numpy as np, pandas as pd
from sklearn.preprocessing import LabelEncoder, PowerTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
from sklearn.linear_model import LogisticRegression

SEED = 42
np.random.seed(SEED); random.seed(SEED)

def clip_and_renorm(probs, eps=1e-15):
    P = np.clip(probs, eps, 1 - eps)
    P /= P.sum(axis=1, keepdims=True)
    return P

def load_data():
    train = pd.read_csv('train.csv')
    test = pd.read_csv('test.csv')
    X = train.drop(columns=['id', 'species'])
    y = train['species'].values
    X_test = test.drop(columns=['id'])
    le = LabelEncoder()
    y_enc = le.fit_transform(y)
    classes = le.classes_
    test_ids = test['id'].values
    return X.values, y_enc, X_test.values, classes, test_ids

def train_oof(model, X, y, X_test, n_classes, n_splits=5):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=SEED)
    n = X.shape[0]
    oof = np.zeros((n, n_classes), dtype=np.float64)
    test_pred = np.zeros((X_test.shape[0], n_classes), dtype=np.float64)
    fold_losses = []
    start = time.time()
    from sklearn.base import clone
    for i, (tr, va) in enumerate(skf.split(X, y), 1):
        t0 = time.time()
        clf = clone(model)
        clf.fit(X[tr], y[tr])
        va_proba = clf.predict_proba(X[va])
        loss = log_loss(y[va], va_proba, labels=list(range(n_classes)))
        oof[va] = va_proba
        test_pred += clf.predict_proba(X_test) / n_splits
        print(f'[PT+LR C2000] Fold {i}/{n_splits} logloss={loss:.6f} time={time.time()-t0:.1f}s', flush=True)
        fold_losses.append(loss)
    oof_loss = log_loss(y, oof, labels=list(range(n_classes)))
    print(f'[PT+LR C2000] OOF={oof_loss:.6f} | mean_folds={np.mean(fold_losses):.6f} | total={(time.time()-start)/60:.1f}m', flush=True)
    return oof, test_pred, oof_loss

X, y, X_test, classes, test_ids = load_data()
n_classes = len(classes)
pipe = Pipeline([
    ('pt', PowerTransformer(method='yeo-johnson', standardize=True)),
    ('clf', LogisticRegression(multi_class='multinomial', solver='saga', penalty='l2', C=2000, max_iter=30000, tol=1e-4, n_jobs=-1, random_state=SEED))
])
oof, test_pred, oof_loss = train_oof(pipe, X, y, X_test, n_classes, n_splits=5)
test_pred = clip_and_renorm(test_pred)
sub = pd.DataFrame(test_pred, columns=classes)
sub.insert(0, 'id', test_ids)
sub.to_csv('submission.csv', index=False)
print('Saved submission.csv with shape:', sub.shape)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


[PT+LR C2000] Fold 1/5 logloss=0.032064 time=48.4s


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


[PT+LR C2000] Fold 2/5 logloss=0.035834 time=44.9s


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


...

[PT+LR C2000] Fold 4/5 logloss=0.043918 time=47.3s


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


[PT+LR C2000] Fold 5/5 logloss=0.031508 time=46.3s


[PT+LR C2000] OOF=0.035058 | mean_folds=0.035062 | total=3.9m


Saved submission.csv with shape: (99, 100)


In [ ]:
# Add Nystroem RBF feature map + Logistic Regression; blend with PT+LR
import time
from sklearn.kernel_approximation import Nystroem
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
from sklearn.pipeline import Pipeline
import numpy as np
import pandas as pd

SEED = 42

def train_oof_fast(model, X, y, X_test, n_classes, n_splits=5, desc='model'):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=SEED)
    n = X.shape[0]
    oof = np.zeros((n, n_classes), dtype=np.float64)
    test_pred = np.zeros((X_test.shape[0], n_classes), dtype=np.float64)
    fold_losses = []
    start = time.time()
    from sklearn.base import clone
    for i, (tr, va) in enumerate(skf.split(X, y), 1):
        t0 = time.time()
        clf = clone(model)
        clf.fit(X[tr], y[tr])
        va_proba = clf.predict_proba(X[va])
        loss = log_loss(y[va], va_proba, labels=list(range(n_classes)))
        oof[va] = va_proba
        test_pred += clf.predict_proba(X_test) / n_splits
        print(f'[{desc}] Fold {i}/{n_splits} logloss={loss:.6f} time={time.time()-t0:.1f}s', flush=True)
        fold_losses.append(loss)
    oof_loss = log_loss(y, oof, labels=list(range(n_classes)))
    print(f'[{desc}] OOF={oof_loss:.6f} | mean_folds={np.mean(fold_losses):.6f} | total={(time.time()-start)/60:.1f}m', flush=True)
    return oof, test_pred, oof_loss

# Preserve LR results from previous cell
lr_oof, lr_test, lr_loss = oof, test_pred, oof_loss

# Small grid for Nystroem+LR (fast, diverse)
configs = []
for gamma in [1e-4, 3e-4]:
    for n_comp in [800, 1200]:
        for C in [3.0, 10.0]:
            configs.append({'gamma': gamma, 'n_components': n_comp, 'C': C})

best_nys = None
best_oof = None
best_test = None
best_loss = float('inf')
for i, p in enumerate(configs, 1):
    print(f'[Nystroem+LR] {i}/{len(configs)} params={p}', flush=True)
    pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('nys', Nystroem(kernel='rbf', gamma=p['gamma'], n_components=p['n_components'], random_state=SEED)),
        ('clf', LogisticRegression(solver='saga', multi_class='multinomial', C=p['C'], penalty='l2', max_iter=10000, tol=1e-4, n_jobs=-1, random_state=SEED))
    ])
    oof2, test2, loss2 = train_oof_fast(pipe, X, y, X_test, n_classes, n_splits=5, desc=f'NysRBF_C{p["C"]}_g{p["gamma"]}_m{p["n_components"]}')
    if loss2 < best_loss:
        best_loss = loss2; best_oof = oof2; best_test = test2; best_nys = p
print('Best Nystroem+LR:', best_nys, 'OOF=', best_loss)

# Simple blends: equal and inverse-loss; pick best
def clip_and_renorm(probs, eps=1e-15):
    P = np.clip(probs, eps, 1 - eps)
    P /= P.sum(axis=1, keepdims=True)
    return P

bases = [('LR_PT_C2000', lr_oof, lr_test, lr_loss), ('NysRBF_LR', best_oof, best_test, best_loss)]
losses = np.array([b[3] for b in bases], dtype=np.float64)
oofs = [b[1] for b in bases]
tests = [b[2] for b in bases]

# Equal
w_eq = np.array([0.5, 0.5], dtype=np.float64)
oof_eq = clip_and_renorm((oofs[0] * w_eq[0] + oofs[1] * w_eq[1]))
loss_eq = log_loss(y, oof_eq, labels=list(range(n_classes)))

# Inverse-loss
w_il = 1.0 / np.maximum(losses, 1e-9); w_il = w_il / w_il.sum()
oof_il = clip_and_renorm(oofs[0] * w_il[0] + oofs[1] * w_il[1])
loss_il = log_loss(y, oof_il, labels=list(range(n_classes)))

print(f'[Blend check] equal={loss_eq:.6f} | inv-loss={loss_il:.6f} | LR={lr_loss:.6f} | Nys={best_loss:.6f}', flush=True)
w_best = w_il if loss_il < loss_eq else w_eq
test_blend = clip_and_renorm(tests[0] * w_best[0] + tests[1] * w_best[1])

# Save submission
sub = pd.DataFrame(test_blend, columns=classes)
sub.insert(0, 'id', test_ids)
sub.to_csv('submission.csv', index=False)
print('Saved submission.csv (LR + Nystroem+LR blend) with shape:', sub.shape, '| weights=', w_best)

In [ ]:
# CatBoost multiclass (5-fold CV) + blend with strong PT+LR(C=2000)
import sys, subprocess, importlib, time, numpy as np, pandas as pd

def ensure_pkg(pkg):
    try:
        return importlib.import_module(pkg)
    except ImportError:
        print(f'Installing {pkg}...', flush=True)
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', '--no-cache-dir', '--prefer-binary', pkg])
        return importlib.import_module(pkg)

catboost = ensure_pkg('catboost')
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss

SEED = 42

def cb_oof_cv(X_np, y_np, Xte_np, n_classes, params_list, n_splits=5):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=SEED)
    best = {'loss': float('inf')}
    for i, prm in enumerate(params_list, 1):
        print(f'[CatBoost] Config {i}/{len(params_list)}: {prm}', flush=True)
        n = X_np.shape[0]
        oof = np.zeros((n, n_classes), dtype=np.float64)
        test_pred = np.zeros((Xte_np.shape[0], n_classes), dtype=np.float64)
        folds = []
        start = time.time()
        for f, (tr, va) in enumerate(skf.split(X_np, y_np), 1):
            t0 = time.time()
            clf = CatBoostClassifier(loss_function='MultiClass',
                                     depth=prm['depth'],
                                     learning_rate=prm['learning_rate'],
                                     l2_leaf_reg=prm['l2_leaf_reg'],
                                     iterations=5000,
                                     random_seed=SEED,
                                     verbose=False)
            clf.fit(X_np[tr], y_np[tr], eval_set=(X_np[va], y_np[va]), use_best_model=True, early_stopping_rounds=200)
            va_proba = clf.predict_proba(X_np[va])
            loss = log_loss(y_np[va], va_proba, labels=list(range(n_classes)))
            oof[va] = va_proba
            test_pred += clf.predict_proba(Xte_np) / n_splits
            folds.append(loss)
            print(f'[CatBoost] Fold {f}/{n_splits} logloss={loss:.6f} time={time.time()-t0:.1f}s', flush=True)
        oof_loss = log_loss(y_np, oof, labels=list(range(n_classes)))
        print(f'[CatBoost] OOF={oof_loss:.6f} | mean_folds={np.mean(folds):.6f} | total={(time.time()-start)/60:.1f}m', flush=True)
        if oof_loss < best.get('loss', float('inf')):
            best = {'params': prm, 'loss': oof_loss, 'oof': oof, 'test': test_pred}
    return best

# Ensure we have LR results from Cell 0
lr_oof, lr_test, lr_loss = oof, test_pred, oof_loss

# Small, conservative CatBoost grid per expert advice
cb_params = [
    {'depth': 6, 'learning_rate': 0.03, 'l2_leaf_reg': 10},
    {'depth': 6, 'learning_rate': 0.06, 'l2_leaf_reg': 10},
    {'depth': 8, 'learning_rate': 0.03, 'l2_leaf_reg': 10},
    {'depth': 8, 'learning_rate': 0.06, 'l2_leaf_reg': 10}
]
best_cb = cb_oof_cv(X, y, X_test, len(classes), cb_params, n_splits=5)
print('Best CatBoost:', best_cb.get('params'), 'OOF=', best_cb.get('loss'))

def clip_and_renorm(probs, eps=1e-15):
    P = np.clip(probs, eps, 1 - eps)
    P /= P.sum(axis=1, keepdims=True)
    return P

# Simple blends: equal and inverse-loss with LR
oofs = [lr_oof, best_cb['oof']]; tests = [lr_test, best_cb['test']]; losses = np.array([lr_loss, best_cb['loss']], dtype=np.float64)
w_eq = np.array([0.5, 0.5]);
oof_eq = clip_and_renorm(oofs[0]*w_eq[0] + oofs[1]*w_eq[1])
loss_eq = log_loss(y, oof_eq, labels=list(range(len(classes))))
w_il = 1.0 / np.maximum(losses, 1e-9); w_il = w_il / w_il.sum()
oof_il = clip_and_renorm(oofs[0]*w_il[0] + oofs[1]*w_il[1])
loss_il = log_loss(y, oof_il, labels=list(range(len(classes))))
print(f'[Blend LR+CB] equal={loss_eq:.6f} | invloss={loss_il:.6f} | LR={lr_loss:.6f} | CB={best_cb["loss"]:.6f}', flush=True)
w_best = w_il if loss_il < loss_eq else w_eq
test_blend = clip_and_renorm(tests[0]*w_best[0] + tests[1]*w_best[1])
sub = pd.DataFrame(test_blend, columns=classes)
sub.insert(0, 'id', test_ids)
sub.to_csv('submission.csv', index=False)
print('Saved submission.csv (LR + CatBoost blend) with shape:', sub.shape, '| weights=', w_best)

In [ ]:
# LightGBM multiclass (raw features) conservative CV with early stopping
import sys, subprocess, importlib, time, numpy as np, pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss

def ensure_pkg(pkg):
    import importlib
    try:
        return importlib.import_module(pkg)
    except ImportError:
        print(f'Installing {pkg}...', flush=True)
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', '--no-cache-dir', '--prefer-binary', pkg])
        return importlib.import_module(pkg)

lgb = ensure_pkg('lightgbm')
from lightgbm import LGBMClassifier

SEED = 42

# Use RAW X, y, X_test from earlier load_data() (no transforms)
n_classes = len(classes)

lgbm_params = {
    'objective': 'multiclass',
    'num_class': n_classes,
    'metric': 'multi_logloss',
    'learning_rate': 0.02,
    'n_estimators': 5000,
    'num_leaves': 12,
    'max_depth': 6,
    'min_child_samples': 28,
    'feature_fraction': 0.7,
    'subsample': 0.8,
    'lambda_l2': 5.0,
    'random_state': SEED,
    'n_jobs': -1,
    'verbosity': -1
}

def lgbm_oof_cv(X_np, y_np, Xte_np, params, n_splits=5, early_rounds=200):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=SEED)
    n = X_np.shape[0]
    oof = np.zeros((n, n_classes), dtype=np.float64)
    test_pred = np.zeros((Xte_np.shape[0], n_classes), dtype=np.float64)
    fold_losses, fold_best_iters = [], []
    start = time.time()
    for f, (tr, va) in enumerate(skf.split(X_np, y_np), 1):
        t0 = time.time()
        clf = LGBMClassifier(**params)
        clf.fit(X_np[tr], y_np[tr],
                eval_set=[(X_np[va], y_np[va])],
                eval_metric='multi_logloss',
                callbacks=[lgb.early_stopping(stopping_rounds=early_rounds, verbose=False)])
        va_proba = clf.predict_proba(X_np[va], raw_score=False)
        loss = log_loss(y_np[va], va_proba, labels=list(range(n_classes)))
        oof[va] = va_proba
        test_pred += clf.predict_proba(Xte_np, raw_score=False) / n_splits
        best_it = getattr(clf, 'best_iteration_', None)
        fold_best_iters.append(best_it if best_it is not None else params.get('n_estimators', 0))
        fold_losses.append(loss)
        print(f'[LGBM] Fold {f}/{n_splits} logloss={loss:.6f} best_it={best_it} time={time.time()-t0:.1f}s', flush=True)
        # Early abort heuristic if poor after two folds
        if f >= 2 and np.mean(fold_losses) > 0.06:
            print('[LGBM] Early abort: mean fold loss above 0.06 after two folds', flush=True)
            break
    used = len(fold_losses)
    if used != n_splits and used > 0:
        test_pred *= (n_splits / used)
    oof_loss = log_loss(y_np, oof, labels=list(range(n_classes)))
    mbest = np.mean([i for i in fold_best_iters if i is not None]) if fold_best_iters else float('nan')
    print(f'[LGBM] OOF={oof_loss:.6f} | mean_folds={np.mean(fold_losses) if fold_losses else float("nan"):.6f} | folds_used={used}/{n_splits} | mean_best_it={mbest:.1f} | total={(time.time()-start)/60:.1f}m', flush=True)
    return oof, test_pred, oof_loss, fold_losses

lgbm_oof, lgbm_test, lgbm_loss, lgbm_fold_losses = lgbm_oof_cv(X, y, X_test, lgbm_params, n_splits=5, early_rounds=200)

def clip_and_renorm(probs, eps=1e-15):
    P = np.clip(probs, eps, 1 - eps)
    P /= P.sum(axis=1, keepdims=True)
    return P

# Quick sanity blend with existing LR OOF/test if present
try:
    lr_oof, lr_test, lr_loss
    oofs = [lr_oof, lgbm_oof]
    tests = [lr_test, lgbm_test]
    losses = np.array([lr_loss, lgbm_loss], dtype=np.float64)
    w_il = 1.0 / np.maximum(losses, 1e-9); w_il = w_il / w_il.sum()
    oof_blend = clip_and_renorm(oofs[0]*w_il[0] + oofs[1]*w_il[1])
    loss_blend = log_loss(y, oof_blend, labels=list(range(n_classes)))
    print(f'[LGBM+LR] inv-loss blend OOF={loss_blend:.6f} | LR={lr_loss:.6f} | LGBM={lgbm_loss:.6f} | weights={w_il}', flush=True)
    test_blend = clip_and_renorm(tests[0]*w_il[0] + tests[1]*w_il[1])
    sub = pd.DataFrame(test_blend, columns=classes)
    sub.insert(0, 'id', test_ids)
    sub.to_csv('submission.csv', index=False)
    print('Saved submission.csv (LR + LGBM inv-loss blend) with shape:', sub.shape, flush=True)
except NameError:
    print('LR results not found in kernel; skipping quick blend. You can blend later.', flush=True)

In [ ]:
# Seed-bagged PT(YJ)+LR: multiple CV seeds and C values; average probs
import time, numpy as np, pandas as pd, random
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
from sklearn.preprocessing import PowerTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

def run_pt_lr_cv(X_np, y_np, Xte_np, n_classes, cv_seed=42, C=2000.0, n_splits=5):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=cv_seed)
    oof = np.zeros((X_np.shape[0], n_classes), dtype=np.float64)
    test_pred = np.zeros((Xte_np.shape[0], n_classes), dtype=np.float64)
    fold_losses = []
    start = time.time()
    for f, (tr, va) in enumerate(skf.split(X_np, y_np), 1):
        t0 = time.time()
        pipe = Pipeline([
            ('pt', PowerTransformer(method='yeo-johnson', standardize=True)),
            ('clf', LogisticRegression(multi_class='multinomial', solver='lbfgs', penalty='l2', C=C, max_iter=30000, tol=1e-5, random_state=cv_seed))
        ])
        pipe.fit(X_np[tr], y_np[tr])
        proba_va = pipe.predict_proba(X_np[va])
        loss = log_loss(y_np[va], proba_va, labels=list(range(n_classes)))
        oof[va] = proba_va
        test_pred += pipe.predict_proba(Xte_np) / n_splits
        fold_losses.append(loss)
        print(f'[Bag PT+LR lbfgs] seed={cv_seed} C={C} fold {f}/{n_splits} loss={loss:.6f} time={time.time()-t0:.1f}s', flush=True)
    oof_loss = log_loss(y_np, oof, labels=list(range(n_classes)))
    print(f'[Bag PT+LR lbfgs] seed={cv_seed} C={C} OOF={oof_loss:.6f} mean_folds={np.mean(fold_losses):.6f} total={(time.time()-start)/60:.1f}m', flush=True)
    return oof, test_pred, oof_loss

def clip_and_renorm(probs, eps=1e-15):
    P = np.clip(probs, eps, 1 - eps)
    P /= P.sum(axis=1, keepdims=True)
    return P

# Configs: single fast run to produce lr_bag_* quickly
seeds = [42]
Cs = [2000.0]
configs = [(s, c) for s in seeds for c in Cs]
print('Total configs:', len(configs), configs, flush=True)

bag_oofs = []
bag_tests = []
bag_losses = []

for i, (s, c) in enumerate(configs, 1):
    print(f'=== Config {i}/{len(configs)}: seed={s}, C={c} ===', flush=True)
    oof_i, test_i, loss_i = run_pt_lr_cv(X, y, X_test, n_classes, cv_seed=s, C=c, n_splits=5)
    bag_oofs.append(oof_i)
    bag_tests.append(test_i)
    bag_losses.append(loss_i)

# Average across bag members
avg_oof = clip_and_renorm(np.mean(bag_oofs, axis=0))
avg_test = clip_and_renorm(np.mean(bag_tests, axis=0))
avg_loss = log_loss(y, avg_oof, labels=list(range(n_classes)))
print(f'[Bag PT+LR lbfgs] Averaged OOF={avg_loss:.6f} | single best={np.min(bag_losses):.6f} | single mean={np.mean(bag_losses):.6f}', flush=True)

# Persist for later stacking/blending
lr_bag_oof = avg_oof
lr_bag_test = avg_test
lr_bag_loss = avg_loss

# Save bagged LR submission
sub = pd.DataFrame(avg_test, columns=classes)
sub.insert(0, 'id', test_ids)
sub.to_csv('submission.csv', index=False)
print('Saved submission.csv (Bagged PT+LR lbfgs) with shape:', sub.shape, flush=True)

In [ ]:
# Fallback: alias single LR OOF/test as bagged outputs if bagging not completed
if 'lr_bag_oof' not in globals() or 'lr_bag_test' not in globals():
    print('Bagged LR not available; using single PT+LR(C=2000) outputs as fallback.', flush=True)
    lr_bag_oof = oof.copy()
    lr_bag_test = test_pred.copy()
    lr_bag_loss = oof_loss

In [2]:
# Ensemble (weighted blend) + Temperature Scaling, then submission
import numpy as np, pandas as pd
from sklearn.metrics import log_loss

def clip_and_renorm(probs, eps=1e-15):
    P = np.clip(probs, eps, 1 - eps)
    P /= P.sum(axis=1, keepdims=True)
    return P

# Fallback: if bagged LR not available, use single strong PT+LR outputs
if 'lr_bag_oof' not in globals() or 'lr_bag_test' not in globals():
    print('Bagged LR not available; using single PT+LR(C=2000) outputs as fallback.', flush=True)
    lr_bag_oof = oof.copy()
    lr_bag_test = test_pred.copy()
    lr_bag_loss = oof_loss

# Collect bases (LR bag; optionally LGBM if available)
bases_oof = [lr_bag_oof]
bases_test = [lr_bag_test]
labels = ['LR_bag']
if 'lgbm_oof' in globals() and 'lgbm_test' in globals():
    bases_oof.append(lgbm_oof)
    bases_test.append(lgbm_test)
    labels.append('LGBM')

K = len(bases_oof)
print('Ensembling bases:', labels, flush=True)

# Optimize 2-model weight w in [0,1]; if only LR, w=1
if K == 1:
    best_w = np.array([1.0])
    oof_blend = clip_and_renorm(bases_oof[0])
    test_blend = clip_and_renorm(bases_test[0])
else:
    ws = np.linspace(0.0, 1.0, 101, dtype=np.float64)
    best_w = None; best_loss = float('inf'); oof_blend = None; test_blend = None
    for w in ws:
        oof_tmp = clip_and_renorm(w * bases_oof[0] + (1.0 - w) * bases_oof[1])
        loss = log_loss(y, oof_tmp, labels=list(range(n_classes)))
        if loss < best_loss:
            best_loss = loss; best_w = np.array([w, 1.0 - w]); oof_blend = oof_tmp
    test_blend = clip_and_renorm(best_w[0] * bases_test[0] + best_w[1] * bases_test[1])
    print(f'[Blend] best_w={best_w} OOF={best_loss:.6f}', flush=True)

# Temperature scaling via power transform: p^alpha then renorm; alpha in (0, +inf).
def apply_temp_scaling(P, alpha):
    Pp = np.power(np.clip(P, 1e-15, 1.0), alpha)
    Pp /= Pp.sum(axis=1, keepdims=True)
    return Pp

# Grid search alpha in [0.5, 2.0]
alphas = np.linspace(0.5, 2.0, 61)
best_alpha = 1.0; best_cal_loss = log_loss(y, oof_blend, labels=list(range(n_classes)))
for a in alphas:
    oof_cal = apply_temp_scaling(oof_blend, a)
    loss = log_loss(y, oof_cal, labels=list(range(n_classes)))
    if loss < best_cal_loss:
        best_cal_loss = loss; best_alpha = a
print(f'[TempScale] best_alpha={best_alpha:.3f} OOF={best_cal_loss:.6f}', flush=True)

# Apply to test
test_final = apply_temp_scaling(test_blend, best_alpha)

# Save submission
sub = pd.DataFrame(test_final, columns=classes)
sub.insert(0, 'id', test_ids)
sub.to_csv('submission.csv', index=False)
print('Saved submission.csv (Ensembled + TempScaled) with shape:', sub.shape, flush=True)

Bagged LR not available; using single PT+LR(C=2000) outputs as fallback.


Ensembling bases: ['LR_bag']


[TempScale] best_alpha=1.675 OOF=0.023995


Saved submission.csv (Ensembled + TempScaled) with shape: (99, 100)
